In [1]:
import pandas as pd
import numpy as np

In [2]:
d1 = pd.read_excel('./raw_data/Travelpac 2014.xls', 'Labelled 2014')
d2 = pd.read_excel('./raw_data/Travelpac 2015.xls', 'Labelled 2015')
d3 = pd.read_excel('./raw_data/Travelpac 2016.xls', 'Labelled 2016 Q3')
d4 = pd.read_excel('./raw_data/Travelpac Q1 2017.xls', 'Labelled 2016 Q1')
d5 = pd.read_excel('./raw_data/Travelpac Q2 2017.xls', 'Labelled 2017 Q2')
d6 = pd.read_excel('./raw_data/Travelpac Q3 2017.xls', 'Labelled 2017 Q3')

In [3]:
#Concatenate them together where axis = 0
total_raw = pd.concat([d1, d2, d3, d4, d5, d6], ignore_index=True)


In [4]:
#Find attribute names
print("Columns:"), print(total_raw.columns)

Columns:
Index(['Year', 'quarter', 'ukos', 'mode', 'country', 'purpose', 'package',
       'Age', 'Sex', 'duration', 'visits', 'nights', 'spend', 'sample'],
      dtype='object')


(None, None)

In [5]:
#Rename columns for clarity
total_raw = total_raw.rename(columns={"ukos":"place_of_residence", "visits": "weighted_visits",
        "nights": "weighted_nights", "spend": "weighted_spend", "mode": "mode_of_transport", 
        "package":"holiday_package", "purpose": "purpose_of_travel"})

print("Updated Columns:"),total_raw.columns

Updated Columns:


(None,
 Index(['Year', 'quarter', 'place_of_residence', 'mode_of_transport', 'country',
        'purpose_of_travel', 'holiday_package', 'Age', 'Sex', 'duration',
        'weighted_visits', 'weighted_nights', 'weighted_spend', 'sample'],
       dtype='object'))

In [6]:
#Create a mapping for all countries we want to include in our analysis
#Codes have been taken from data documentation
mapping_country = {
        10: 'Austria',
        11: 'Belgium',
        12: 'Bulgaria',
        13: 'Croatia',
        14: 'Czech Republic',
        15: 'Cyprus EU',
        16: 'Cyprus Non EU',
        17: 'Denmark',
        18: 'Estonia',
        19: 'Finland',
        20: 'France',#Includes Monaco
        21: 'Germany',
        22: 'Gibraltar',
        23: 'Greece',
        24: 'Hungary',
        25: 'Iceland',
        26: 'Irish Republic',
        27: 'Italy', #Includes San Marino and Vatican City
        28: 'Latvia',
        29: 'Lithuania',
        30: 'Luxembourg',
        31: 'Malta',
        32: 'Netherlands',
        33: 'Norway',
        34: 'Poland',
        35: 'Portugal',#Includes Azores and Madeira
        36: 'Romania',
        37: 'Russia',
        38: 'Spain',#Includes Andorra, Canary Islands
        39: 'Slovakia',
        40: 'Slovenia',
        41: 'Sweden',
        42: 'Switzerland',#Includes Liechtenstein
        43: 'Turkey',
        44: 'Channel Islands',
        45: 'Other Europe',#Table does not specify further 
        }

In [7]:
#Swap keys and values in mapping_country so string name is the key
mapping_country_inv = {v: k for k, v in mapping_country.items()}
#Convert countries to numerical codes
total_raw.replace(mapping_country_inv, inplace = True)

In [8]:
#Quickly create a list of numbers
country_range = list(set(np.arange(10,46,1)))
#remove countries which are not in range 
target_dataset = total_raw.loc[total_raw['country'].isin(country_range)]

In [9]:
#Return country to String category level
target_dataset = target_dataset.replace(mapping_country)

#clean up the index
find_null = target_dataset.reset_index(drop=True)

In [10]:
#Look at new formation of data, we now have 89833 instances
print(target_dataset['country'].describe())

count      89833
unique        36
top       France
freq       11705
Name: country, dtype: object


In [11]:
#Summary of null values in the data

#Find out if there are null values in your data
print('Are there any null values:'),print(find_null.isnull().values.any())
#Find out how many - This gives you the breakdown per column
print('How many in each column?'),print(find_null.isnull().sum())
#Gives you total overall
print('In total?'), print(find_null.isnull().sum().sum())

Are there any null values:
True
How many in each column?
Year                     0
quarter                  0
place_of_residence       0
mode_of_transport        0
country                  0
purpose_of_travel        0
holiday_package          0
Age                      0
Sex                    188
duration                 0
weighted_visits       2536
weighted_nights       2536
weighted_spend           5
sample                 180
dtype: int64
In total?
5445


(None, None)

In [12]:
#Looking at the data, many of the weighted_visits and wieghted_nights comes where Transit is purpose
#We find when we sum by purpose that Transit is NaN
grouped = find_null['weighted_visits'].groupby(find_null['purpose_of_travel']).sum()
print(grouped)
grouped = find_null['weighted_nights'].groupby(find_null['purpose_of_travel']).sum()
print(grouped)

purpose_of_travel
9                1.161210e+03
Business         4.706254e+07
Holiday          1.719312e+08
Miscellaneous    1.169497e+07
Study            1.329115e+06
Transit                   NaN
VFR              7.083480e+07
Name: weighted_visits, dtype: float64
purpose_of_travel
9                4.064235e+05
Business         1.679381e+08
Holiday          1.296363e+09
Miscellaneous    8.590032e+07
Study            4.969528e+07
Transit                   NaN
VFR              6.406660e+08
Name: weighted_nights, dtype: float64


In [13]:
#edit dataframe so that it does not contain rows of Transit or 9
#9 codes as "Not Known" in documentation and only has a count for 1 in our data
remove_transit = find_null[~find_null['purpose_of_travel'].isin(['Transit', 9])]

#Now lets see the amount of null values per column
print("How many in each column?"),print(remove_transit.isnull().sum())
#Only 504 left
print("In total?"), print(remove_transit.isnull().sum().sum())

print(remove_transit.shape)

How many in each column?
Year                    0
quarter                 0
place_of_residence      0
mode_of_transport       0
country                 0
purpose_of_travel       0
holiday_package         0
Age                     0
Sex                   188
duration                0
weighted_visits        64
weighted_nights        64
weighted_spend          5
sample                180
dtype: int64
In total?
501
(87360, 14)


In [14]:
#Drop sample, as it is not needed for our analysis

drop_sample = remove_transit.drop(['sample'], axis=1)

In [15]:
#Deal with NaN values for Sex

#Drop where Sex is not Binary Male/Female
drop_sample = drop_sample[~drop_sample['Sex'].isin(["Dont Know", "Don't Know/NA"])]

#We still have 180 in Gender left. We can't take an average, so:
print("How many NaN do we have left?"),print(drop_sample.isnull().sum())
#Drop NA values from this column
drop_sample.dropna(subset=['Sex'], inplace = True)

How many NaN do we have left?
Year                    0
quarter                 0
place_of_residence      0
mode_of_transport       0
country                 0
purpose_of_travel       0
holiday_package         0
Age                     0
Sex                   188
duration                0
weighted_visits        64
weighted_nights        64
weighted_spend          5
dtype: int64


In [16]:
print("Count of remaining NaN?"),print(drop_sample.isnull().sum())

Count of remaining NaN?
Year                  0
quarter               0
place_of_residence    0
mode_of_transport     0
country               0
purpose_of_travel     0
holiday_package       0
Age                   0
Sex                   0
duration              0
weighted_visits       4
weighted_nights       4
weighted_spend        5
dtype: int64


(None, None)

In [17]:
#Deal with rows with several NaN values. All rows with several NaN values
# have a "Stay not known" and thus no values for visits, nights and sppend either.

#Drop all rows with a value of "Stay Not Known" in column "duration"
remove_stayNotKnown = drop_sample[~drop_sample['duration'].isin(['Stay Not Known'])]

#Now lets see the amount of null values per column
print("How many in each column?"),print(remove_stayNotKnown.isnull().sum())
#Only 1 left
print("In total?"), print(remove_stayNotKnown.isnull().sum().sum())


How many in each column?
Year                  0
quarter               0
place_of_residence    0
mode_of_transport     0
country               0
purpose_of_travel     0
holiday_package       0
Age                   0
Sex                   0
duration              0
weighted_visits       0
weighted_nights       0
weighted_spend        1
dtype: int64
In total?
1


(None, None)

In [18]:
#Deal with NaN value in weighted_spend

#We replace the one missing value with 0 as this is the safest value to assume
final = remove_stayNotKnown.fillna(0)

#Now lets see the amount of null values per column, no NaN values left
print("How many in each column?"),print(final.isnull().sum())

How many in each column?
Year                  0
quarter               0
place_of_residence    0
mode_of_transport     0
country               0
purpose_of_travel     0
holiday_package       0
Age                   0
Sex                   0
duration              0
weighted_visits       0
weighted_nights       0
weighted_spend        0
dtype: int64


(None, None)

In [19]:
#Find Uknown Occurences in our data 

print('Sex unknown occurences: ', (final['Sex']=='Dont know').sum())
print('Age unknown occurences: ', (final['Age']=='D/K').sum())            

#drop unknown age
drop_dkage = final[final['Age']!='D/K']
print('Age unknown occurences now: ', (drop_dkage['Age']=='D/K').sum())
print('Sex unknown occurences now: ', (drop_dkage['Sex']=='Dont know').sum())

#removing the rest of 'Dont know' values from Sex 
drop_dksex = drop_dkage[drop_dkage['Sex']!='Dont know']
print('Age unknown occurences new: ', (drop_dksex['Age']=='D/K').sum())
print('Sex unknown occurences new: ', (drop_dksex['Sex']=='Dont know').sum())

Sex unknown occurences:  70
Age unknown occurences:  137
Age unknown occurences now:  0
Sex unknown occurences now:  14
Age unknown occurences new:  0
Sex unknown occurences new:  0


In [20]:
#Saving the data:

#Reindex 
drop_dksex.reset_index(drop=True, inplace = True)

print(drop_dksex.shape)

(86859, 13)


In [21]:
clean_duration = drop_dksex

In [22]:
print(clean_duration.columns)
#There are problems with the categories for duration
print(clean_duration['duration'].value_counts())
print(clean_duration['duration'].describe())

Index(['Year', 'quarter', 'place_of_residence', 'mode_of_transport', 'country',
       'purpose_of_travel', 'holiday_package', 'Age', 'Sex', 'duration',
       'weighted_visits', 'weighted_nights', 'weighted_spend'],
      dtype='object')
4 - 13 Nights      25274
1 - 3 Nights       21658
4-13 nights         9312
14 - 27 Nights      8481
1-3 nights          8068
Nil Stay            5506
28 - 90 Nights      3524
14-27 nights        2911
28-90 nights        1234
3 - 6 Months         514
3-6 months           193
6 Months - Year      124
6 months-year         60
Name: duration, dtype: int64
count             86859
unique               13
top       4 - 13 Nights
freq              25274
Name: duration, dtype: object


In [33]:
clean_duration.replace({"Nil Stay": "0 Nights", "1-3 nights": "1 - 3 Nights", "4-13 nights": "4 - 13 Nights",
                       "14-27 nights": "14 - 27 Nights", "28-90 nights": "28 - 90 Nights", "3-6 months": "3 - 6 Months",
                       "6 months-year": "6 Months - Year"}, inplace = True)


print(clean_duration['duration'].value_counts())
print(clean_duration['duration'].describe())

4 - 13 Nights      34586
1 - 3 Nights       29726
14 - 27 Nights     11392
0 Nights            5506
28 - 90 Nights      4758
3 - 6 Months         707
6 Months - Year      184
Name: duration, dtype: int64
count             86859
unique                7
top       4 - 13 Nights
freq              34586
Name: duration, dtype: object


C:\Users\hatti\Anaconda2\envs\py36\lib\site-packages\pandas\core\generic.py:3836: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  limit=limit, regex=regex)


In [24]:

print(clean_duration.shape)

clean_duration.tail()

(86859, 13)


,Year,quarter,place_of_residence,mode_of_transport,country,purpose_of_travel,holiday_package,Age,Sex,duration,weighted_visits,weighted_nights,weighted_spend
86854,2017,Jul-Sep,Overseas residents,Tunnel,Switzerland,Holiday,Non-Independent,65 & over,Female,4 - 13 Nights,1124.887,7874.209,833541.267
86855,2017,Jul-Sep,Overseas residents,Tunnel,Switzerland,Business,Independent,35-44,Male,1 - 3 Nights,1251.385,1251.385,508062.310
86856,2017,Jul-Sep,Overseas residents,Tunnel,Switzerland,VFR,Independent,25-34,Male,1 - 3 Nights,1251.385,1251.385,250277.000
86857,2017,Jul-Sep,Overseas residents,Tunnel,Turkey,Business,Independent,55-64,Male,4 - 13 Nights,1226.617,4906.468,24532.340
86858,2017,Jul-Sep,Overseas residents,Tunnel,Other Europe,Business,Independent,35-44,Male,4 - 13 Nights,3233.306,16166.530,0.000


In [25]:
print(clean_duration.isnull().values.any())

False


In [26]:
#splitting a column and writing to a new file
data3 =  clean_duration


In [27]:
#UK Residents
UK_Residents = data3[data3['place_of_residence'].str.contains("UK")]
print(UK_Residents.shape)

#Overseas Residents
Overseas_Residents = data3[data3['place_of_residence'].str.contains("Overseas")]
print(Overseas_Residents.shape)

(45770, 13)
(41089, 13)


In [28]:
#Write to file
writerUK = pd.ExcelWriter('Travelpac_cleaned_UK_final.xlsx')
UK_Residents.to_excel(writerUK, 'Labelled 2014-2017', index=True)
writerUK.save()

In [29]:
#Write to file
writerOverseas = pd.ExcelWriter('Travelpac_cleaned_overseas_final.xlsx')
Overseas_Residents.to_excel(writerOverseas, 'Labelled 2014-2017', index=True)
writerOverseas.save()